# Optimization - January 2018

## Data

### Demand

In [1]:
import pandas as pd
import numpy as np
import csv
demand = pd.read_csv('JanWeeklyDemand.csv')
demand2 = pd.read_csv('JanWeeklyDemand.csv')
demand2['Inventory Date'] = pd.to_datetime(demand2['Inventory Date']).astype('datetime64[D]')
demand = demand.fillna(0)
demand['Inventory Date'] = pd.to_datetime(demand['Inventory Date']).astype('datetime64[D]')
#
demand['ItemCode'] = demand['ItemCode'].str.lstrip('0')
demand2['ItemCode'] = demand2['ItemCode'].str.lstrip('0')
#
demand = demand.set_index([demand['Inventory Date'],demand['ItemCode'],demand['Branch']])
demand = demand.drop(['Inventory Date','ItemCode','Branch'], axis = 1)
demand = demand.T
columnlist = []
for c in demand.columns:
    x, y, z = c
    x = np.datetime64(x)
    x = x.astype('datetime64[D]')
    if y[0:2] == 'SR':
        y = y[2:]
        c2 = (x,y,z)
    else:
        c2 = (x,y,z)
    columnlist.append(c2)
demand.columns = columnlist
demand = demand.groupby(by = demand.columns,axis=1).sum()
# Example selection: 
# demand[weeks[1],'271','Toronto F&S']['Weekly Demand']

In [2]:
demand

,"(2018-01-07, 0200600, Toronto F&S)","(2018-01-07, 0240600, Toronto F&S)","(2018-01-07, 0241000, Toronto F&S)","(2018-01-07, 0241200, Toronto F&S)","(2018-01-07, 0241250, Toronto F&S)","(2018-01-07, 0241600, Edmonton F&S)","(2018-01-07, 0241600, Toronto F&S)","(2018-01-07, 0241601, Toronto F&S)","(2018-01-07, 0241608, Toronto F&S)","(2018-01-07, 0241616, Toronto F&S)",...,"(2018-01-28, Y412XWB, Baltimore F&S)","(2018-01-28, Y412XWB, Edmonton F&S)","(2018-01-28, Y412XWB, Fontana)","(2018-01-28, Y412XWB, Great Lakes CC)","(2018-01-28, Y412XWB, Houston F&S)","(2018-01-28, Y412XWB, Linden)","(2018-01-28, Y412XWB, Sacramento Main)","(2018-01-28, Y412XWB, Seattle F&S)","(2018-01-28, Y412XWB, Tampa F&S)","(2018-01-28, Y412XWB, Vancouver F&S)"
Weekly Demand,1,18,60,1786,5386,16,4433,176,706,120,...,20,9,15,35,69,12,33,1,9,17


### Branch-to-Branch Cost

In [3]:
# costs = pd.read_csv('RandomCosts.csv')
# costs['Cost'].apply(lambda x: float(x))
# #
# costs['Item'] = costs['Item'].str.lstrip('0')
# #
# costs = costs.set_index(['From','To','Item'])
# costs = costs.T.to_dict()
# # Example of selection: 
# #costs['Atlanta F&S','Atlanta F&S','200600']['Cost']

In [4]:
regression={'Mileage':0.3328,'Weight':0.09408,'OLinden':- 99.8,'OBaltimore F&S':179.3,'OAtlanta F&S':258.7,'OTampa F&S':111.0,
           'O43302':325.1,'OGreat Lakes CC':335.3,'OHouston F&S':245.2,'OFontana':- 55.1,'OSacramento Main':- 218.4,'OSeattle F&S':38.9,'OToronto F&S':532.6,
           'DLinden':-17.8,'DBaltimore F&S':59.1,'DAtlanta F&S':-138.2,'DTampa F&S':107.7,
           'D43302':190.2,'DGreat Lakes CC':- 5.6,'DHouston F&S':- 348.5,'DFontana':431.5,'DSacramento Main':558.7,'DSeattle F&S':635.2,'DToronto F&S':407.6,
           'MM':- 0.000060,'WW':- 0.000002,'MW':0.000041}

import csv
reader = csv.reader(open('weights.csv', 'r',encoding = 'utf8'))
weights = {}
for row in reader:
    x, item,y, weight = row
    weights[item] = weight

miles = {}

with open("distance_df.csv", "r") as infile:
    reader = csv.reader(infile)
    headers = next(reader)[1:]
    for row in reader:
        miles[row[0]] = {key: float(value) for key, value in zip(headers, row[1:])}
def cost(i,j,k):
    jx=j
    kx=k
    if i not in weights.keys():
        if '0'+i in weights.keys():
            i='0'+i
        
        elif 'SR'+i in weights.keys():
            i='SR'+i
            
        else:
            print(i)
            weights[i]=1
    if jx in ['Edmonton F&S','Vancouver F&S','Montreal F&S']:
        jx='Toronto F&S'
    if kx in ['Edmonton F&S','Vancouver F&S','Montreal F&S']:
        kx='Toronto F&S'
    c=(float(weights[i])/40000)*(-286.0 + 0.3328*miles[k][j]+regression['O'+jx]+regression['D'+kx]+0.09408*float(40000)- 0.000060*(miles[k][j])**2 - 0.000002*float(40000)**2+0.000041*float(40000)*miles[k][j])
    return c

### Total Units in System

In [5]:
totaldata = pd.read_csv('2018InventoryTotals.csv')
#
nosr = []
totaldata['ItemCode'] = totaldata['ItemCode'].str.lstrip('0')
for c in totaldata['ItemCode']:
    if c[0:2] == 'SR':
        c2 = c[2:]
    else:
        c2 = c
    nosr.append(c2)
totaldata['ItemCode'] = nosr    
totaldata = totaldata.groupby(by = 'ItemCode').sum()
# Example selection: totaldata['TotalUnits']['4004']

### Dealing with Missing Data

In [6]:
w = pd.read_csv('DiscrepanciesNoSR2.csv')
w.set_index('Item Code')
bad = w['Item Code']
bad = bad.tolist()
bad

['10795',
 '1121',
 '1133',
 '1579',
 '1579',
 '1579',
 '1579',
 '1991487',
 '1991487',
 '2053',
 '2053',
 '2053',
 '2053',
 '2190',
 '2190',
 '2190',
 '2190',
 '2199',
 '2199',
 '2199',
 '2199',
 '3561',
 '3561',
 '404',
 '4765',
 '526',
 '5321',
 '7018',
 '7018',
 '7018',
 '7018',
 '7230264',
 '7230264',
 '7230264',
 '7230455',
 '7230455',
 '7230455',
 '7471',
 '7471',
 '7471',
 'AL1S',
 'ALT3',
 'LB10904',
 'LB10905',
 'LB10908',
 'MESH2X4',
 'SLH22',
 '100852',
 '100852',
 '225006',
 '225006',
 '225016',
 '225016',
 '225016',
 '404',
 'ALJ21',
 'ALJ21',
 'B10',
 'B10',
 'B10',
 'B10',
 'SFSJ36',
 'SFSJ36',
 'SLH22',
 'SRB',
 'SRB',
 'SRO',
 'SRO',
 'VFTP463087',
 'VFTP463087',
 'VFTP552147',
 'VFTP552147',
 'VFTP552147',
 'VFTP552147',
 '10795',
 '1121',
 '1133',
 '1579',
 '1991487',
 '2053',
 '2190',
 '2199',
 '3561',
 '404',
 '4765',
 '526',
 '5321',
 '7018',
 '7230264',
 '7230455',
 '7471',
 'AL1S',
 'ALT3',
 'LB10904',
 'LB10905',
 'LB10908',
 'MESH2X4',
 'SLH22',
 '100852',
 '

## Gurobi Model

In [7]:
from gurobipy import GRB, Model, quicksum
m = Model('AlumaJan')

Academic license - for non-commercial use only


In [8]:
plist = demand2['ItemCode'].unique()
partlist=[]
for p in plist:
    if p[0:2] == 'SR':
        p2 = p[2:]
    else:
        p2 = p
    partlist.append(p2.lstrip('0'))
partlist = pd.Series(partlist)
partlist = partlist.unique()
partlist = list(filter(lambda a: a not in bad,partlist))

In [9]:
branchlist = demand2['Branch'].unique()
branchlist = pd.Series(branchlist)
branchlist = branchlist.sort_values()
weeks = demand2['Inventory Date'].unique()
weeks = weeks.astype('datetime64[D]')

In [10]:
indexlist = []
for part in partlist:
    for frombranch in branchlist:
        for tobranch in branchlist:
            for week in weeks:
                indexlist.append((part,frombranch,tobranch,week))
indexlist2 = []
for part in partlist:
    for branch in branchlist:
        for week in weeks:
            indexlist2.append((part,branch,week))

In [11]:
x = m.addVars(indexlist, name ='x')
y = m.addVars(indexlist2, name = "y")

### Constraining y values when t = 1
***

In [12]:
y1 = pd.read_csv('2018 - January 7 Inventory Totals.csv')
y1['InventoryDate'] = weeks[0]
nosr2 = []
for g in y1['ItemCode']:
    g = str(g)
    if g[0:2] == 'SR':
        g2 = g[2:]
    else:
        g2 = g
    nosr2.append(g2)
y1['ItemCode'] = nosr2 
y1 = y1[y1.Branch != 'USD Region Offices']
y1 = y1.groupby(by = ['ItemCode','Branch','InventoryDate']).sum()
# Example Selection: y1['InventoryQuantity']['11','Atlanta F&S', weeks[0]]
for idx,r in y1.iterrows():
    if idx in indexlist2:
        try:
            m.addConstr(y[idx[0],idx[1],weeks[0]] == np.maximum(r, demand[weeks[0],idx[0],idx[1]]['Weekly Demand'])) #y1['InventoryQuantity'][idx[0],idx[1],weeks[0]])
        except KeyError:
            m.addConstr(y[idx[0],idx[1],weeks[0]] == r)      
    else:
        pass
                               

***

In [13]:
for i in partlist:
    for j in branchlist:
        for t in weeks:
            m.addConstrs(x[i,j,k,t] == 0 for k in branchlist if k == j)
            try:
                m.addConstr(y[i,j,t] >= demand[t,i,j]['Weekly Demand'])
            except KeyError:
                pass
            m.addConstr(y[i,j,t] >= 0)
        for t in weeks[0:len(weeks)-1]:
            try:
                m.addConstr(y[i,j,t+np.timedelta64(7,'D')] == y[i,j,t] + 
                            quicksum(x[i,k,j,t] for k in branchlist) - 
                            quicksum(x[i,j,k,t+np.timedelta64(7,'D')] for k in branchlist))
            except KeyError:
                pass
           
for i in partlist:
        for t in weeks:
            try:
                m.addConstr(quicksum(y[i,g,t] for g in branchlist) + quicksum(x[i,q,w,t] for q in branchlist for w in branchlist) <= totaldata['TotalUnits'][i])
            except KeyError:
                pass

In [14]:
m.setObjective(quicksum(cost(i,j,k)*
                        x[i,j,k,t]
                        for i in partlist
                        for j in branchlist
                        for k in branchlist
                        for t in weeks),
               GRB.MINIMIZE)

2488US
289
4014
531
K908730


In [15]:
m.optimize()
#m.computeIIS()
#m.write("model.ilp")

Optimize a model with 413678 rows, 1889160 columns and 3146213 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-05, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 410362 rows and 1883698 columns (presolve time = 5s) ...
Presolve removed 410362 rows and 1883698 columns
Presolve time: 6.66s
Presolved: 3316 rows, 5462 columns, 9455 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.993e+03
 Factor NZ  : 9.808e+03 (roughly 4 MBytes of memory)
 Factor Ops : 3.739e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.91040398e+08  1.26861451e+04  1.22e+05 0.00e+00  8.99e+05     7s
   1   3.76739718e+08 -1.49895696e+06  1.05e+04 1.12e+01  2.76e+05     7s
   2   1.57023638e+08 -5

In [16]:
status_code = {1:'LOADED', 2:'OPTIMAL', 3:'INFEASIBLE', 4:'INF_OR_UNBD', 5:'UNBOUNDED'}
status = m.status
               
print('The optimization status is {}'.format(status_code[status]))
if status == 2:  
     print('Optimal solution:')
     print('Optimal objective value:\n{}'.format(m.objVal))
     for v in m.getVars():
            if v.x != 0:
                print('%s = %g' % (v.varName, v.x))
     

The optimization status is OPTIMAL
Optimal solution:
Optimal objective value:
28464.209533368943
x[10624,Fontana,Sacramento Main,2018-01-07] = 200
x[11,Linden,Great Lakes CC,2018-01-14] = 528
x[115,Linden,Montreal F&S,2018-01-14] = 3
x[12,Sacramento Main,Fontana,2018-01-07] = 1348
x[125,Linden,Baltimore F&S,2018-01-07] = 18
x[125,Linden,Baltimore F&S,2018-01-21] = 28
x[126,Sacramento Main,Fontana,2018-01-07] = 92
x[131,Fontana,Sacramento Main,2018-01-07] = 15
x[131,Fontana,Sacramento Main,2018-01-21] = 6
x[136,Fontana,Sacramento Main,2018-01-07] = 50
x[137,Atlanta F&S,Tampa F&S,2018-01-07] = 10
x[139,Linden,Montreal F&S,2018-01-07] = 2
x[14,Sacramento Main,Fontana,2018-01-07] = 856
x[16,Sacramento Main,Fontana,2018-01-07] = 42
x[1605,Seattle F&S,Vancouver F&S,2018-01-07] = 3
x[1633,Linden,Baltimore F&S,2018-01-21] = 548
x[164,Linden,Baltimore F&S,2018-01-14] = 20
x[164,Linden,Baltimore F&S,2018-01-21] = 28
x[164,Vancouver F&S,Seattle F&S,2018-01-07] = 4
x[1718003,Atlanta F&S,Tampa F&S,

x[745,Linden,Great Lakes CC,2018-01-07] = 28
x[7541,Linden,Great Lakes CC,2018-01-21] = 243
x[7551,Atlanta F&S,Houston F&S,2018-01-07] = 56
x[7562,Vancouver F&S,Seattle F&S,2018-01-21] = 4
x[7601,Atlanta F&S,Tampa F&S,2018-01-14] = 199
x[7863,Fontana,Sacramento Main,2018-01-07] = 70
x[7951,Atlanta F&S,Tampa F&S,2018-01-07] = 925
x[8103,Fontana,Sacramento Main,2018-01-07] = 17
x[8166,Atlanta F&S,Tampa F&S,2018-01-07] = 211
x[8211,Atlanta F&S,Tampa F&S,2018-01-07] = 44
x[85,Sacramento Main,Fontana,2018-01-07] = 186
x[85,Seattle F&S,Vancouver F&S,2018-01-07] = 5
x[8641,Sacramento Main,Fontana,2018-01-07] = 1
x[87,Sacramento Main,Fontana,2018-01-07] = 2
x[870,Linden,Great Lakes CC,2018-01-14] = 2
x[8824,Vancouver F&S,Seattle F&S,2018-01-07] = 4
x[8854,Atlanta F&S,Tampa F&S,2018-01-07] = 10
x[8920,Sacramento Main,Fontana,2018-01-14] = 7
x[899,Vancouver F&S,Seattle F&S,2018-01-07] = 3
x[900,Linden,Great Lakes CC,2018-01-07] = 26
x[910,Linden,Great Lakes CC,2018-01-14] = 2
x[920,Linden,Great 

y[6K,Fontana,2018-01-28] = 1480
y[6K,Great Lakes CC,2018-01-07] = 249
y[6K,Great Lakes CC,2018-01-14] = 249
y[6K,Great Lakes CC,2018-01-21] = 249
y[6K,Great Lakes CC,2018-01-28] = 249
y[6K,Houston F&S,2018-01-07] = 244
y[6K,Houston F&S,2018-01-14] = 244
y[6K,Houston F&S,2018-01-21] = 244
y[6K,Houston F&S,2018-01-28] = 244
y[6K,Linden,2018-01-07] = 66
y[6K,Linden,2018-01-14] = 66
y[6K,Linden,2018-01-21] = 66
y[6K,Linden,2018-01-28] = 66
y[6K,Sacramento Main,2018-01-07] = 541
y[6K,Sacramento Main,2018-01-14] = 541
y[6K,Sacramento Main,2018-01-21] = 541
y[6K,Sacramento Main,2018-01-28] = 541
y[6K,Seattle F&S,2018-01-07] = 525
y[6K,Seattle F&S,2018-01-14] = 525
y[6K,Seattle F&S,2018-01-21] = 525
y[6K,Seattle F&S,2018-01-28] = 525
y[6K,Tampa F&S,2018-01-07] = 52
y[6K,Tampa F&S,2018-01-14] = 52
y[6K,Tampa F&S,2018-01-21] = 52
y[6K,Tampa F&S,2018-01-28] = 52
y[6K,Vancouver F&S,2018-01-07] = 80
y[6K,Vancouver F&S,2018-01-14] = 80
y[6K,Vancouver F&S,2018-01-21] = 80
y[6K,Vancouver F&S,2018-01-2

y[10684,Atlanta F&S,2018-01-28] = 46
y[10684,Great Lakes CC,2018-01-07] = 69
y[10684,Great Lakes CC,2018-01-14] = 69
y[10684,Great Lakes CC,2018-01-21] = 69
y[10684,Great Lakes CC,2018-01-28] = 69
y[10684,Houston F&S,2018-01-07] = 16
y[10684,Houston F&S,2018-01-14] = 16
y[10684,Houston F&S,2018-01-21] = 16
y[10684,Houston F&S,2018-01-28] = 16
y[10684,Linden,2018-01-07] = 644
y[10684,Linden,2018-01-14] = 644
y[10684,Linden,2018-01-21] = 644
y[10684,Linden,2018-01-28] = 644
y[10684,Sacramento Main,2018-01-07] = 83
y[10684,Sacramento Main,2018-01-14] = 83
y[10684,Sacramento Main,2018-01-21] = 83
y[10684,Sacramento Main,2018-01-28] = 83
y[10684,Tampa F&S,2018-01-07] = 80
y[10684,Tampa F&S,2018-01-14] = 80
y[10684,Tampa F&S,2018-01-21] = 80
y[10684,Tampa F&S,2018-01-28] = 80
y[10696,Atlanta F&S,2018-01-07] = 4387
y[10696,Atlanta F&S,2018-01-14] = 4387
y[10696,Atlanta F&S,2018-01-21] = 4387
y[10696,Atlanta F&S,2018-01-28] = 4387
y[10696,Baltimore F&S,2018-01-07] = 806
y[10696,Baltimore F&S,2

y[112,Baltimore F&S,2018-01-28] = 4421
y[112,Edmonton F&S,2018-01-07] = 3290
y[112,Edmonton F&S,2018-01-14] = 3290
y[112,Edmonton F&S,2018-01-21] = 3290
y[112,Edmonton F&S,2018-01-28] = 3290
y[112,Fontana,2018-01-07] = 2737
y[112,Fontana,2018-01-14] = 2737
y[112,Fontana,2018-01-21] = 2737
y[112,Fontana,2018-01-28] = 2737
y[112,Great Lakes CC,2018-01-07] = 506
y[112,Great Lakes CC,2018-01-14] = 506
y[112,Great Lakes CC,2018-01-21] = 506
y[112,Great Lakes CC,2018-01-28] = 506
y[112,Houston F&S,2018-01-07] = 2859
y[112,Houston F&S,2018-01-14] = 2859
y[112,Houston F&S,2018-01-21] = 2859
y[112,Houston F&S,2018-01-28] = 2859
y[112,Montreal F&S,2018-01-07] = 2859
y[112,Montreal F&S,2018-01-14] = 2859
y[112,Montreal F&S,2018-01-21] = 2859
y[112,Montreal F&S,2018-01-28] = 2859
y[112,Sacramento Main,2018-01-07] = 1
y[112,Sacramento Main,2018-01-14] = 1
y[112,Sacramento Main,2018-01-21] = 1
y[112,Sacramento Main,2018-01-28] = 1
y[112,Seattle F&S,2018-01-07] = 1579
y[112,Seattle F&S,2018-01-14] = 

...

y[M61000,Houston F&S,2018-01-07] = 24
y[M61000,Houston F&S,2018-01-14] = 24
y[M61000,Houston F&S,2018-01-21] = 24
y[M61000,Houston F&S,2018-01-28] = 24
y[M61000,Linden,2018-01-07] = 71
y[M61000,Linden,2018-01-14] = 71
y[M61000,Linden,2018-01-21] = 71
y[M61000,Linden,2018-01-28] = 71
y[M61000,Tampa F&S,2018-01-07] = 36
y[M61000,Tampa F&S,2018-01-14] = 36
y[M61000,Tampa F&S,2018-01-21] = 36
y[M61000,Tampa F&S,2018-01-28] = 36
y[M61400,Atlanta F&S,2018-01-07] = 68
y[M61400,Atlanta F&S,2018-01-14] = 68
y[M61400,Atlanta F&S,2018-01-21] = 68
y[M61400,Atlanta F&S,2018-01-28] = 68
y[M61400,Houston F&S,2018-01-07] = 172
y[M61400,Houston F&S,2018-01-14] = 172
y[M61400,Houston F&S,2018-01-21] = 172
y[M61400,Houston F&S,2018-01-28] = 172
y[M61400,Linden,2018-01-07] = 130
y[M61400,Linden,2018-01-14] = 130
y[M61400,Linden,2018-01-21] = 130
y[M61400,Linden,2018-01-28] = 130
y[M61400,Sacramento Main,2018-01-07] = 30
y[M61400,Sacramento Main,2018-01-14] = 30
y[M61400,Sacramento Main,2018-01-21] = 30
y[

y[PSH1116D,Fontana,2018-01-21] = 4826
y[PSH1116D,Fontana,2018-01-28] = 4826
y[PSH1116D,Great Lakes CC,2018-01-07] = 120
y[PSH1116D,Great Lakes CC,2018-01-14] = 120
y[PSH1116D,Great Lakes CC,2018-01-21] = 120
y[PSH1116D,Great Lakes CC,2018-01-28] = 120
y[PSH1116D,Houston F&S,2018-01-07] = 1047
y[PSH1116D,Houston F&S,2018-01-14] = 1047
y[PSH1116D,Houston F&S,2018-01-21] = 1047
y[PSH1116D,Houston F&S,2018-01-28] = 1047
y[PSH1116D,Linden,2018-01-07] = 555
y[PSH1116D,Linden,2018-01-14] = 555
y[PSH1116D,Linden,2018-01-21] = 555
y[PSH1116D,Linden,2018-01-28] = 555
y[PSH1116D,Sacramento Main,2018-01-07] = 7626
y[PSH1116D,Sacramento Main,2018-01-14] = 7626
y[PSH1116D,Sacramento Main,2018-01-21] = 7626
y[PSH1116D,Sacramento Main,2018-01-28] = 7626
y[PSH1116D,Seattle F&S,2018-01-07] = 5895
y[PSH1116D,Seattle F&S,2018-01-14] = 5895
y[PSH1116D,Seattle F&S,2018-01-21] = 5895
y[PSH1116D,Seattle F&S,2018-01-28] = 5895
y[PSH1118,Baltimore F&S,2018-01-07] = 36
y[PSH1118,Baltimore F&S,2018-01-14] = 36
y[

y[Q2STAX2,Linden,2018-01-21] = 11
y[Q2STAX2,Linden,2018-01-28] = 11
y[Q2STAX3,Great Lakes CC,2018-01-07] = 136
y[Q2STAX3,Great Lakes CC,2018-01-14] = 136
y[Q2STAX3,Great Lakes CC,2018-01-21] = 136
y[Q2STAX3,Great Lakes CC,2018-01-28] = 136
y[Q2STAX3,Linden,2018-01-07] = 9
y[Q2STAX3,Linden,2018-01-14] = 9
y[Q2STAX3,Linden,2018-01-21] = 9
y[Q2STAX3,Linden,2018-01-28] = 9
y[Q2STAX4,Great Lakes CC,2018-01-07] = 26
y[Q2STAX4,Great Lakes CC,2018-01-14] = 26
y[Q2STAX4,Great Lakes CC,2018-01-21] = 26
y[Q2STAX4,Great Lakes CC,2018-01-28] = 26
y[Q2STAX4,Linden,2018-01-07] = 7
y[Q2STAX4,Linden,2018-01-14] = 7
y[Q2STAX4,Linden,2018-01-21] = 7
y[Q2STAX4,Linden,2018-01-28] = 7
y[Q2STSX1L,Fontana,2018-01-07] = 51
y[Q2STSX1L,Fontana,2018-01-14] = 51
y[Q2STSX1L,Fontana,2018-01-21] = 51
y[Q2STSX1L,Fontana,2018-01-28] = 51
y[Q2STSX1L,Great Lakes CC,2018-01-07] = 9
y[Q2STSX1L,Great Lakes CC,2018-01-14] = 9
y[Q2STSX1L,Great Lakes CC,2018-01-21] = 9
y[Q2STSX1L,Great Lakes CC,2018-01-28] = 9
y[Q2STSX1L,Sacra

y[SLBC,Toronto F&S,2018-01-14] = 556
y[SLBC,Toronto F&S,2018-01-21] = 556
y[SLBC,Toronto F&S,2018-01-28] = 556
y[SLDH70,Edmonton F&S,2018-01-07] = 12
y[SLDH70,Edmonton F&S,2018-01-14] = 12
y[SLDH70,Edmonton F&S,2018-01-21] = 12
y[SLDH70,Edmonton F&S,2018-01-28] = 12
y[SLH10,Edmonton F&S,2018-01-07] = 166
y[SLH10,Edmonton F&S,2018-01-14] = 166
y[SLH10,Edmonton F&S,2018-01-21] = 166
y[SLH10,Edmonton F&S,2018-01-28] = 166
y[SLH10,Houston F&S,2018-01-07] = 12
y[SLH10,Houston F&S,2018-01-14] = 12
y[SLH10,Houston F&S,2018-01-21] = 12
y[SLH10,Houston F&S,2018-01-28] = 12
y[SLH10,Linden,2018-01-07] = 71
y[SLH10,Linden,2018-01-14] = 71
y[SLH10,Linden,2018-01-21] = 71
y[SLH10,Linden,2018-01-28] = 71
y[SLH10,Toronto F&S,2018-01-07] = 112
y[SLH10,Toronto F&S,2018-01-14] = 112
y[SLH10,Toronto F&S,2018-01-21] = 112
y[SLH10,Toronto F&S,2018-01-28] = 112
y[SLH30,Montreal F&S,2018-01-07] = 3
y[SLH30,Montreal F&S,2018-01-14] = 3
y[SLH30,Montreal F&S,2018-01-21] = 3
y[SLH30,Montreal F&S,2018-01-28] = 3
y

y[ST12,Great Lakes CC,2018-01-14] = 47
y[ST12,Great Lakes CC,2018-01-21] = 47
y[ST12,Great Lakes CC,2018-01-28] = 47
y[ST12SG,Fontana,2018-01-07] = 4
y[ST12SG,Fontana,2018-01-14] = 4
y[ST12SG,Fontana,2018-01-21] = 4
y[ST12SG,Fontana,2018-01-28] = 4
y[ST13SG,Atlanta F&S,2018-01-07] = 12
y[ST13SG,Atlanta F&S,2018-01-14] = 12
y[ST13SG,Atlanta F&S,2018-01-21] = 12
y[ST13SG,Atlanta F&S,2018-01-28] = 12
y[ST13SG,Baltimore F&S,2018-01-07] = 442
y[ST13SG,Baltimore F&S,2018-01-14] = 442
y[ST13SG,Baltimore F&S,2018-01-21] = 442
y[ST13SG,Baltimore F&S,2018-01-28] = 442
y[ST13SG,Edmonton F&S,2018-01-07] = 933
y[ST13SG,Edmonton F&S,2018-01-14] = 933
y[ST13SG,Edmonton F&S,2018-01-21] = 933
y[ST13SG,Edmonton F&S,2018-01-28] = 933
y[ST13SG,Fontana,2018-01-07] = 891
y[ST13SG,Fontana,2018-01-14] = 891
y[ST13SG,Fontana,2018-01-21] = 891
y[ST13SG,Fontana,2018-01-28] = 891
y[ST13SG,Houston F&S,2018-01-07] = 20
y[ST13SG,Houston F&S,2018-01-14] = 20
y[ST13SG,Houston F&S,2018-01-21] = 20
y[ST13SG,Houston F&S,